In [25]:
import pandas as pd

In [26]:
df = pd.read_csv("data/persona.csv")

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [28]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PRICE,5000.0,34.1320,12.464897,9.0,29.0,39.0,39.0,59.0
AGE,5000.0,23.5814,8.995908,15.0,17.0,21.0,27.0,66.0


In [29]:
df.SOURCE.nunique()

2

In [30]:
df.SOURCE.value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [31]:
df.groupby("SOURCE")["PRICE"].mean()

SOURCE
android    34.174849
ios        34.069102
Name: PRICE, dtype: float64

In [32]:
df.PRICE.nunique()

6

In [33]:
df.PRICE.value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [34]:
df.COUNTRY.value_counts()

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [35]:
df.groupby("COUNTRY")["PRICE"].sum()

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

In [36]:
df.groupby("COUNTRY")["PRICE"].mean()

COUNTRY
bra    34.327540
can    33.608696
deu    34.032967
fra    33.587459
tur    34.787140
usa    34.007264
Name: PRICE, dtype: float64

In [37]:
df.groupby(["COUNTRY", "SOURCE"])["PRICE"].mean()

COUNTRY  SOURCE 
bra      android    34.387029
         ios        34.222222
can      android    33.330709
         ios        33.951456
deu      android    33.869888
         ios        34.268817
fra      android    34.312500
         ios        32.776224
tur      android    36.229437
         ios        33.272727
usa      android    33.760357
         ios        34.371703
Name: PRICE, dtype: float64

In [38]:
df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"])["PRICE"].mean()

COUNTRY  SOURCE   SEX     AGE
bra      android  female  15     38.714286
                          16     35.944444
                          17     35.666667
                          18     32.255814
                          19     35.206897
                                   ...    
usa      ios      male    42     30.250000
                          50     39.000000
                          53     34.000000
                          55     29.000000
                          59     46.500000
Name: PRICE, Length: 348, dtype: float64

Aggregation

In [39]:
agg_df = (
    df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"])["PRICE"]
    .mean()
    .sort_values(ascending=False)
)
agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


Age Categories

In [40]:
agg_df["AGE_CAT"] = pd.cut(
    x=agg_df["AGE"],
    bins=[0, 18, 23, 30, 40, 66, 100],
    labels=["0_18", "19_23", "24_30", "31_40", "41_66", "67_100"],
)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


Feature Engineering

In [41]:
agg_df["customers_level_based"] = (
    agg_df[["COUNTRY", "SOURCE", "SEX", "AGE_CAT"]]
    .apply(lambda x: x.str.upper())
    .agg("_".join, axis=1)
)
agg_df = (
    agg_df.groupby("customers_level_based")["PRICE"].mean().reset_index()
)  # to remove duplicates
agg_df.sort_values("PRICE", ascending=False, inplace=True)
agg_df.reset_index(inplace=True, drop=True)
agg_df.head()

,customers_level_based,PRICE
0,FRA_ANDROID_FEMALE_24_30,45.428571
1,TUR_IOS_MALE_24_30,45.000000
2,TUR_IOS_MALE_31_40,42.333333
3,TUR_ANDROID_FEMALE_31_40,41.833333
4,CAN_ANDROID_MALE_19_23,40.111111


Segmentation

In [42]:
agg_df.describe().T

,count,mean,std,min,25%,50%,75%,max
PRICE,109.0,34.056925,4.102984,19.0,32.333333,34.07734,36.0,45.428571


In [44]:
agg_df["SEGMENT"] = pd.qcut(agg_df.PRICE, 4, labels=["D", "C", "B", "A"])
agg_df.head()

,customers_level_based,PRICE,SEGMENT
0,FRA_ANDROID_FEMALE_24_30,45.428571,A
1,TUR_IOS_MALE_24_30,45.000000,A
2,TUR_IOS_MALE_31_40,42.333333,A
3,TUR_ANDROID_FEMALE_31_40,41.833333,A
4,CAN_ANDROID_MALE_19_23,40.111111,A


In [45]:
agg_df["SEGMENT"].value_counts()

D    28
C    27
B    27
A    27
Name: SEGMENT, dtype: int64

In [48]:
agg_df.drop("customers_level_based", axis=1).groupby("SEGMENT").agg(
    ["mean", "max", "sum"]
)

PRICE                        
              mean        max          sum
SEGMENT                                   
D        29.206780  32.333333   817.789833
C        33.509674  34.077340   904.761209
B        34.999645  36.000000   944.990411
A        38.691234  45.428571  1044.663328

In [49]:
agg_df[agg_df.SEGMENT == "C"].describe().T

,count,mean,std,min,25%,50%,75%,max
PRICE,27.0,33.509674,0.492587,32.5,33.0,33.627634,34.0,34.07734


Querying

In [50]:
user1 = "TUR_ANDROID_FEMALE_31_40"
user2 = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df.customers_level_based == user1][["PRICE", "SEGMENT"]]
agg_df[agg_df.customers_level_based == user2][["PRICE", "SEGMENT"]]

,PRICE,SEGMENT
78,32.818182,C
